Генерируем аннотации при помощи модели mBart. Замеряем ROUGE

In [1]:
from transformers import MBartTokenizer, MBartForConditionalGeneration


In [2]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [3]:
model = model.cuda()

In [4]:
import torch

In [5]:
!nvidia-smi

Wed Apr 28 16:33:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P2   137W / 350W |   5318MiB / 24265MiB |     60%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
torch.cuda.is_available()

True

In [7]:
print(model)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=T

In [8]:
GAZETA_PATH = '../data/gazeta_jsonl'

In [9]:
import json

In [10]:
def read_gazeta_records(file_name, shuffle=False, sort_by_date=True):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle(records)
    return records

In [11]:
import os

In [12]:
dataset_files = {
    'train': os.path.join(GAZETA_PATH,'gazeta_train.jsonl'),
    'val': os.path.join(GAZETA_PATH,'gazeta_val.jsonl'),
    'test': os.path.join(GAZETA_PATH, 'gazeta_test.jsonl')
}

In [13]:
records = {
    split: read_gazeta_records(path) for split, path in dataset_files.items()
}

In [14]:
article_text = records['val'][0]['text']

In [15]:
article_text

'Будущее капитана московского «Спартака» Дениса Глушакова весь этот сезон находится в подвешенном состоянии, и, похоже, развязка уже близка. Красно-белых ждет серьезная перестройка в летнее трансферное окно, и новому генеральному директору команды Томасу Цорну поставили задачу максимально выгодно расстаться с некоторыми футболистами — в том числе и с 32-летним полузащитником, который ощутимо сдал по игровым кондициям и впал в немилость у большинства фанатов клуба из-за скандальной ссоры с Массимо Каррерой. Однако по контракту у Глушакова еще остается год игры за «Спартак» с зарплатой 3 млн евро в год, как сообщает «СЭ». В случае досрочного расторжения клуб должен будет выплатить футболисту половину этой суммы. И вот тут начинается самое интересное. Адвокат бывшей жены Глушакова Дарьи — Сергей Жорин — заявил, что ему кажется «очень похожей на правду» информация о том, что футболист якобы попросил клуб отдать ему 1,5 млн наличными, чтобы избежать выплаты алиментов. Напомним, суд обязал ф

In [16]:
input_ids = tokenizer.prepare_seq2seq_batch(
    [article_text],
    src_lang="en_XX", # fairseq training artifact
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=600
)["input_ids"]

/home/ivan/Programming/Python/general37/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


In [17]:
# input_ids

In [18]:
import random

In [19]:
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids.cuda(),
        max_length=162,
        no_repeat_ngram_size=3,
        num_beams=10,
        top_k=0
    )[0]

In [20]:
summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [21]:
print(summary)

Защитник московского «Спартака» Денис Глушаков заявил, что ему кажется «очень похожей на правду» информация о том, что футболист якобы попросил клуб отдать ему 1,5 млн наличными, чтобы избежать выплаты алиментов.


In [22]:
from rouge import Rouge

In [23]:
def calc_metrics(refs, hyps, metric="all"):
    metrics = dict()
    metrics["count"] = len(hyps)
    metrics["ref_example"] = refs[-1]
    metrics["hyp_example"] = hyps[-1]

    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(hyps, refs, avg=True)
        metrics.update(scores)

    return metrics

In [24]:
def print_metrics(refs, hyps, metric="all"):
    metrics = calc_metrics(refs, hyps, metric=metric)

    print("-------------METRICS-------------")
    print("Count:\t", metrics["count"])
    print("Ref:\t", metrics["ref_example"])
    print("Hyp:\t", metrics["hyp_example"])

#     if "bleu" in metrics:
#         print("BLEU:     \t{:3.1f}".format(metrics["bleu"] * 100.0))
    if "rouge-1" in metrics:
#         print([metrics["rouge-1"][m] * 100.0 for m in ('p','r','f')])
        print("ROUGE-1: P: {:3.2f} R: {:3.2f} F: {:3.2f}".format(
            *[metrics["rouge-1"][m] * 100.0 for m in ['p','r','f']]))
        print("ROUGE-2: P: {:3.2f} R: {:3.2f} F: {:3.2f}".format(
            *[metrics["rouge-2"][m] * 100.0 for m in ['p','r','f']]))
        print("ROUGE-L: P: {:3.2f} R: {:3.2f} F: {:3.2f}".format(
            *[metrics["rouge-l"][m] * 100.0 for m in ['p','r','f']]))


In [25]:
import razdel

In [26]:
def postprocess(refs, hyps, tokenize_after=True, lower=True):
    for i, (ref, hyp) in enumerate(zip(refs, hyps)):
        ref = ref.strip()
        hyp = hyp.strip()
        if tokenize_after:
            hyp = " ".join([token.text for token in razdel.tokenize(hyp)])
            ref = " ".join([token.text for token in razdel.tokenize(ref)])
        if lower:
            hyp = hyp.lower()
            ref = ref.lower()
        refs[i] = ref
        hyps[i] = hyp
    return refs, hyps

In [27]:
from tqdm.notebook import tqdm

In [28]:
def get_model_device(model):
    return next(iter(model.parameters())).device

In [29]:
def calc_method_score(records, predict_func, nrows=None, return_ref_pred=False, text_key='text'):
    references = []
    predictions = []

    for i, record in tqdm(enumerate(records)):
        if nrows is not None and i >= nrows:
            break
        summary = record["summary"]
        text = record[text_key]
        prediction = predict_func(text, summary)
        references.append(summary)
        predictions.append(prediction)
    references, predictions = postprocess(references, predictions)
    print_metrics(references, predictions)
    if return_ref_pred:
        return references, predictions

Функция предсказания аннотаций. Снизьте num_beams для ускорения (может понизить качество). В качестве refsu

In [30]:
def predict_with_bart(text):
    input_ids = tokenizer.prepare_seq2seq_batch(
        [text],
        src_lang="en_XX", # fairseq training artifact
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=600
    )["input_ids"]
    
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids.cuda(),
            max_length=162,
            no_repeat_ngram_size=3,
            num_beams=10,
            top_k=0
        )[0]
        
        
    summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    return summary

In [32]:
print(records['val'][1]['text'])

Сенаторы Соединенных Штатов предлагают ввести пять видов санкционных ограничений против тех, кто страхует суда, укладывающие «Северный поток — 2». Об этом сообщается на сайте конгресса. «Президент может ввести пять или более санкций… в отношении иностранного лица, если он решит, что это лицо осознанно в дату вступления этого закона в силу или после нее предоставит услуги по оценке рисков, страхованию или перестрахованию судну», — указывается в законопроекте. Законопроект предусматривает, что ограничения в первую очередь будут вводиться против иностранных компаний, которые предоставляют свои специализированные суда для укладки газовых труб в Балтийском море. Суда для строительства Nord Stream 2 предоставляют три компании: швейцарская Allseas, итальянская Saipem и российская МРТС («Межрегионтрубопроводстрой»). Помимо этого санкции должны быть введены и против юридических или физических лиц, которые сознательно оказывают страховые или гарантийные услуги этим судам. Законопроект обяжет соб

In [33]:
print(records['val'][1]['summary'])

Ни много ни мало, а пять видов санкций предлагают ввести США против участвующих в российском проекте «Северный поток — 2». Замешанным в СП — 2 запретят любые финансовые операции, в том числе выдачу кредитов. Взамен российского газа США продвигают «молекулы свободы» — свой сжиженный природный газ.


In [34]:
print(predict_with_bart(records['val'][1]['text']))

/home/ivan/Programming/Python/general37/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Глава Минэнерго США Рик Перри заявил, что Вашингтон планирует ввести санкции против компаний, участвующих в строительстве газопровода «Северный поток — 2». Законопроект предусматривает пять видов санкционных ограничений против тех, кто страхует суда, укладывающие газовые трубы в Балтийском море. Авторами законопроекта являются Джин Шахин, Тед Круз, Том Коттон и Джон Баррассо. Черновой вариант законопроекта был подготовлен представителями обеих парламентских партий еще 14 мая, что повышает шансы на принятие документа.


In [35]:
model.training

False

In [36]:
random.seed(4543)

In [37]:
refs, preds = calc_method_score(random.sample(records['val'],70), 
                                lambda x,y: predict_with_bart(x), 
                                return_ref_pred=True)

|          | 0/? [00:00<?, ?it/s]

-------------METRICS-------------
Count:	 70
Ref:	 украинская певица светлана лобода пожаловалась подписчикам на жуткие гематомы . согласно артистке , травмы стали следствием ее концертной деятельности .
Hyp:	 украинская певица светлана лобода поделилась с подписчиками фотографией своих ног , на которых отчетливо видны огромные гематомы . по словам 36-летней уроженки киева , данные травмы стали прямым следствием ее активной концертной деятельности . ранее в сми появилась информация , что лидер немецкой метал-группы rammstein тилль линдеманн сломал челюсть поклоннику из-за девушки .
ROUGE-1: P: 33.43 R: 38.19 F: 34.50
ROUGE-2: P: 14.35 R: 16.64 F: 14.87
ROUGE-L: P: 29.62 R: 33.25 F: 30.46


In [38]:
print(refs[0])

верховный суд ес снял санкции с экс-президента украины виктора януковича и шести его приближенных . судьи решили , что замораживание их счетов и активов было проведено в 2016-2018 годах незаконно . ограничительные меры были сняты по причине отсутствия достаточных доказательств того , что чиновники выводили похищенные государственные средства за рубеж .


In [39]:
print(preds[0])

европейский суд отменил санкции в отношении бывшего президента украины виктора януковича и его окружения , отменив тем самым решение европейского совета по продлению санкций в их отношении . по мнению судей , замораживание счетов и активов названных лиц в 2016-2018 годах было незаконным .


In [40]:
with open('bart_result.txt', 'w+') as f:
    for ref, hyp in zip(refs, preds):
        f.write(ref)
        f.write("\n\n")
        f.write(hyp)
        f.write("\n\n=============\n\n")